# Install and import dependences

In [1]:
#!pip install --upgrade python-binance tensorflow

In [2]:
from binance.client import Client
from binance import BinanceSocketManager
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from sklearn.metrics import mean_squared_error, confusion_matrix
from collections import Counter

# Load data

In [3]:
api_key = "imuSkmeAI4PpxNERZRcnzx6nEkReEhuLedZphxZNphhnUPLR4QH61aXfjnBqUtfH"
api_secret = "56AvdFdOWWoRF20rh6Knq8awDYM145luMDH0XvDMkSD6nffbRwYTCrsxwJmyVLK1"
client = Client(api_key, api_secret)

In [4]:
#Get the exchange info, which contains information about all trading pairs
exchange_info = client.get_exchange_info()

#Extract the symbols of all trading pairs
symbols = [symbol['symbol'] for symbol in exchange_info['symbols'] if "BRL" in symbol['symbol']]

#Filter out only the base cryptocurrencies (e.g., BTC, ETH)
symbols = list(symbol.split(' ')[0] for symbol in symbols)

In [5]:
# Function to fetch historical cryptocurrency data
def fetch_binance_cryptocurrency_data(symbol, start_date, end_date, interval='12h'):
    klines = client.get_historical_klines(symbol, interval, start_date, end_date)

    # Convert data to a Pandas DataFrame
    df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')  # Convert timestamp to datetime

    return df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]

database = {}
present_day = dt.date.today()
start_date = '1900-01-01'
end_date = f"{present_day}"
for ticker in symbols:
    data = fetch_binance_cryptocurrency_data(ticker, start_date, end_date)
    database[ticker] = data.close.values.astype(float)

# Model Functions

In [46]:
def model(price: list, look_back: int = 11, epochs: int = 100, ticker: str = None, plot_graph: bool = True):
    # Prepare data
    X = []
    y = []
    for i in range(len(price) - look_back):
        X.append(price[i:i+look_back])
        y.append(price[i+look_back])
    X = np.array(X)
    y = np.array(y)
    X = X.reshape(X.shape[0], X.shape[1], 1)
      
    # Create & train model
    model = Sequential([
        SimpleRNN(11, activation='relu', input_shape=(look_back, 1), return_sequences=True),
        SimpleRNN(11, activation='relu', return_sequences=True),
        SimpleRNN(11, activation='relu', return_sequences=False),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, y, epochs=epochs, verbose=0)
    
    # Predict   
    X_test = y[-look_back:]
    pred1 = model.predict(X_test.reshape(1,look_back,1), verbose=0)
    new_x_test = pred1[0][0]
    
    X_test = np.append(X_test,new_x_test)
    pred2 = model.predict(X_test[-look_back:].reshape(1,look_back,1), verbose=0)
    
    # Set desired variables
    last_price = y[-1]
    status = last_price < pred1 <= pred2 or (last_price < pred2 < pred1 and (pred1-pred2) <= pred1*0.0025) or (pred1 < last_price < pred2 and (last_price-pred1) <= pred1*0.0025)
    
    if plot_graph:
        # Plot dataset + predictions
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
        ax1.plot(np.append(y_train, y_test), label='True Price')
        ax1.plot(np.arange(len(y_train), len(y_train) + len(pred)), pred, label='Predictions')
        ax1.set_xlabel('Time')
        ax1.set_ylabel('Price')
        ax1.set_title(f'{ticker} Stock Price Prediction')
        ax1.legend()

        ax2.plot(np.append(y_train[-1], y_test), label='True Price')
        ax2.plot(np.arange(len(y_test) - 1, len(y_test) + 1), pred, label='Predictions')
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Price')
        ax2.set_title(f'{ticker} Stock Price Prediction')
        ax2.legend()

        plt.tight_layout()
        plt.show()
    
    return status, last_price, pred1, pred2

# Other Functions

In [74]:
# Suppress the warning related to TensorFlow tracing
tf.autograph.set_verbosity(0)

def pred_eval(model_function, epochs: int = 100, ticker_list: list = symbols, print_info: bool = True):
    buy_list = []
    verify_buy_list = []
    pred_status_list= []
    last_price_list = []
    pred_1_list, pred_2_list = [], []
    count = 0
    total_last_stock_price, total_gamble_stock_price = 0, 0
    total_pred_stock_price = 0
    total_proft = 0

    for ticker in ticker_list:
        count += 1
        price = database[ticker]
        act_status, last_price, pred1, pred2 = model_function(price, look_back = 1, epochs=epochs, ticker=ticker, plot_graph = False)

        pred_status_list.append(act_status)
        last_price_list.append(last_price)
        pred_1_list.append(pred1)
        pred_2_list.append(pred2)

        my_act_status = act_status
        my_gamble = sell_price(last_price, pred1, pred2)
        gamble_proft = my_gamble/last_price

        if my_gamble/last_price < 1.001 and my_act_status == True: 
            my_act_status = False
        if my_act_status == False: 
            gamble_proft = None

        if print_info:
            print('{:<2} '.format(count), 
                  '{:<10} -'.format(ticker), 
                  'LastPrice: {:<10}'.format(round(float(last_price), 3)), 
                  '/ Pred1: {:<10}'.format(round(float(pred1), 3)), 
                  '/ Pred2: {:<10}'.format(round(float(pred2), 3)), 
                  '|| ActStatus: {:<1}'.format(int(act_status)), 
                  '/ MyActStatus: {:<1}'.format(int(my_act_status)), 
                  '/ MyGamble: {:<10}'.format(round(float(my_gamble), 3)), 
                  '/ GambleProft: {:<5}'.format(round(float(gamble_proft), 3) if gamble_proft is not None else ''))

        if act_status:
            total_last_stock_price += last_price
            total_pred_stock_price += pred1
            total_gamble_stock_price += my_gamble

        if act_status and my_act_status:
            buy = ['{:<8}'.format(ticker), 
                   'LastPrice: {:<11}'.format(round(float(last_price), 5)), 
                   'MyGamble: {:<11}'.format(round(float(my_gamble), 5)), 
                   'GambleProft: {:<5}'.format(round(float(gamble_proft), 3) if gamble_proft is not None else '')]
            buy_list.append(buy)
            verify_buy_list.append([ticker, float(last_price), float(my_gamble)])
            
    return verify_buy_list

In [54]:
def my_confusion_matrix(true_status: list, pred_status: list):
    cm = confusion_matrix(true_status, pred_status)
    labels = ['True Negatives', 'False Positives', 'False Negatives', 'True Positives']
    cm = cm.reshape((2, 2))
    
    # Print indices where true and pred statuses are False and True respectively
    false_true_indices = []
    for i in range(len(true_status)):
        if true_status[i] == False and pred_status[i] == True:
            false_true_indices.append(i)
    
    # Calculate accuracy
    accuracy = (cm[0,0] + cm[1,1]) / np.sum(cm)
    print("CM accuracy:", accuracy)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.set(font_scale=1.4)  # Adjust font size
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()
    
    return false_true_indices 

In [55]:
def sell_price(last_value, pred1, pred2, margin = 0.001):
    'Optmize the sell price'
    if pred1 <= last_value and abs(last_value - pred1) <= (pred1*0.0025):
        return last_value
    
    if (pred1 - (pred1*margin)) <= last_value:
        return last_value
    
    if pred2/pred1 >= 1.05:
        if pred1/last_value > 1.02:
            return last_value * 1.02
        if pred1 + (pred1 * margin) <= last_value * 1.02:
            return pred1 + (pred1 * margin)
        else:
            return last_value * 1.02
    
    if pred2/pred1 >= 1.01:
        if pred1/last_value > 1.02:
            return last_value * 1.02
        if pred1 <= last_value * 1.02:
            return pred1
        else: 
            return last_value * 1.02

    else:
        return pred1 - (pred1 * margin)

In [71]:
def refine_gambles(verify_list):
    tickers = []
    last_prices = []
    gambles = []
    
    for iteration in verify_list:
        iter_ticker = []
        iter_gamble = []
        iter_true_price = []
        iter_last_price = []
        
        for crypto in iteration:
            iter_ticker.append(crypto[0])
            iter_last_price.append(crypto[1])
            iter_gamble.append(crypto[2])
            
        tickers.append(iter_ticker)
        last_prices.append(iter_last_price)
        gambles.append(iter_gamble)
    
    all_tickers = [ticker for sublist in tickers for ticker in sublist]   # Flatten the list of lists into a single list
    ticker_counts = Counter(all_tickers)   # Count the occurrences of each ticker symbol
    chosen_cryptos = []
    for ticker, count in ticker_counts.items():
        chosen_cryptos.append([ticker, count])    
    
    avg_gambles = {}
    for chosen in chosen_cryptos:
        if chosen[1] >= len(verify_list)*0.6:
            ticker = chosen[0] 
            total_gamble = 0
            count = 0
            
            for iteration in verify_list:
                for crypto in iteration:
                    if crypto[0] == ticker:
                        count += 1
                        last_price = crypto[1]
                        total_gamble += crypto[2]
                        
            avg_gamble = total_gamble/count
            gamble_proft = avg_gamble/last_price
            avg_gambles[ticker] = [avg_gamble, gamble_proft]

    return avg_gambles

# Predictions & Evaluate

In [61]:
verify_list = []
for i in range(10):
    d = dt.datetime.now()
    verify_buy = pred_eval(model, epochs = 100, lb_list = [1], ticker_list = symbols, print_info = True)
    verify_list.append(verify_buy)
    d2 = dt.datetime.now()
    print(d2-d, '\n\n')
    
gambles = refine_gambles(verify_list)
gambles

1   BTCBRL   - LastPrice: 359839.0   / Pred1: 359439.406 / Pred2: 359040.25  || ActStatus: 0 / MyActStatus: 0 / MyGamble: 359839.0   / GambleProft:      
2   USDTBRL  - LastPrice: 5.021      / Pred1: 5.033      / Pred2: 5.046      || ActStatus: 1 / MyActStatus: 1 / MyGamble: 5.028      / GambleProft: 1.001
3   ETHBRL   - LastPrice: 20317.9    / Pred1: 20600.615  / Pred2: 20887.225  || ActStatus: 1 / MyActStatus: 1 / MyGamble: 20600.615  / GambleProft: 1.014
4   BNBBRL   - LastPrice: 2672.0     / Pred1: 2671.778   / Pred2: 2671.556   || ActStatus: 0 / MyActStatus: 0 / MyGamble: 2672.0     / GambleProft:      
5   BUSDBRL  - LastPrice: 4.991      / Pred1: 5.001      / Pred2: 5.011      || ActStatus: 1 / MyActStatus: 0 / MyGamble: 4.996      / GambleProft:      
6   XRPBRL   - LastPrice: 3.48       / Pred1: 3.466      / Pred2: 3.451      || ActStatus: 0 / MyActStatus: 0 / MyGamble: 3.48       / GambleProft:      
7   LINKBRL  - LastPrice: 105.85     / Pred1: 105.858    / Pred2: 105.865   

In [73]:
verify_list = []
for i in range(10):
    d = dt.datetime.now()
    verify_buy = pred_eval(model, epochs = 100, lb_list = [1], ticker_list = symbols, print_info = True)
    verify_list.append(verify_buy)
    d2 = dt.datetime.now()
    print(d2-d, '\n\n')
    
gambles = refine_gambles(verify_list)
gambles

1   BTCBRL   - LastPrice: 359839.0   / Pred1: 360903.344 / Pred2: 361970.844 || ActStatus: 1 / MyActStatus: 1 / MyGamble: 360542.438 / GambleProft: 1.002
2   USDTBRL  - LastPrice: 5.021      / Pred1: 5.028      / Pred2: 5.035      || ActStatus: 1 / MyActStatus: 0 / MyGamble: 5.023      / GambleProft:      
3   ETHBRL   - LastPrice: 20317.9    / Pred1: 20506.828  / Pred2: 20697.492  || ActStatus: 1 / MyActStatus: 1 / MyGamble: 20486.322  / GambleProft: 1.008
4   BNBBRL   - LastPrice: 2672.0     / Pred1: 2638.693   / Pred2: 2605.826   || ActStatus: 0 / MyActStatus: 0 / MyGamble: 2672.0     / GambleProft:      
5   BUSDBRL  - LastPrice: 4.991      / Pred1: 5.022      / Pred2: 5.05       || ActStatus: 1 / MyActStatus: 1 / MyGamble: 5.017      / GambleProft: 1.005
6   XRPBRL   - LastPrice: 3.48       / Pred1: 3.46       / Pred2: 3.44       || ActStatus: 0 / MyActStatus: 0 / MyGamble: 3.48       / GambleProft:      
7   LINKBRL  - LastPrice: 105.85     / Pred1: 106.246    / Pred2: 106.642   

{'BTCBRL': [361275.2604166667, 1.0039913973100933],
 'ENJBRL': [1.3457476140757971, 1.0133641672257507],
 'HOTBRL': [0.038179772174358376, 1.0181272579828902],
 'CAKEBRL': [69.87859471638997, 1.0022747377565975],
 'AXSBRL': [38.89994607772827, 1.0140757580221136],
 'FISBRL': [1.8043934810147968, 1.0131350258364946],
 'FTMBRL': [1.9466053058242798, 1.012275250038627],
 'MANABRL': [1.6802522827339172, 1.0115907782865246],
 'LUNABRL': [0.0003059999999999999, 1.0199999999999998],
 'APEBRL': [22.31661097208659, 1.0070672821338713],
 'GALBRL': [8.76705259467231, 1.0100291007686992],
 'APTBRL': [39.959368569510325, 1.005014300037986],
 'BNBBRL': [2689.4964192708335, 1.0065480611043538],
 'GMTBRL': [1.341252871831258, 1.0122663183632137]}

In [77]:
verify_list = []
count = 0
for i in range(80):
    count += 1
    print('Count:', count)
    d = dt.datetime.now()
    verify_buy = pred_eval(model, epochs = 100, ticker_list = symbols, print_info = True)
    verify_list.append(verify_buy)
    d2 = dt.datetime.now()
    print(d2-d, '\n\n')
    
gambles = refine_gambles(verify_list)
gambles

1   BTCBRL     - LastPrice: 359839.0   / Pred1: 355875.719 / Pred2: 351956.062 || ActStatus: 0 / MyActStatus: 0 / MyGamble: 359839.0   / GambleProft:      
2   USDTBRL    - LastPrice: 5.021      / Pred1: 5.013      / Pred2: 5.006      || ActStatus: 0 / MyActStatus: 0 / MyGamble: 5.021      / GambleProft:      
3   ETHBRL     - LastPrice: 20317.9    / Pred1: 20356.66   / Pred2: 20395.49   || ActStatus: 1 / MyActStatus: 0 / MyGamble: 20336.303  / GambleProft:      
4   BNBBRL     - LastPrice: 2672.0     / Pred1: 2692.726   / Pred2: 2713.595   || ActStatus: 1 / MyActStatus: 1 / MyGamble: 2690.033   / GambleProft: 1.007
5   BUSDBRL    - LastPrice: 4.991      / Pred1: 4.995      / Pred2: 4.998      || ActStatus: 1 / MyActStatus: 0 / MyGamble: 4.991      / GambleProft:      
6   XRPBRL     - LastPrice: 3.48       / Pred1: 3.517      / Pred2: 3.555      || ActStatus: 1 / MyActStatus: 1 / MyGamble: 3.517      / GambleProft: 1.011
7   LINKBRL    - LastPrice: 105.85     / Pred1: 105.472    / Pre

{'ENJBRL': [1.4128607761104905, 1.0639011868301886],
 'AXSBRL': [38.77345493108575, 1.0107782828750196],
 'SANTOSBRL': [20.201951622962948, 1.0100975811481474],
 'LUNABRL': [0.00030600000000000044, 1.0200000000000016],
 'GALBRL': [8.757469865611244, 1.0089250997247976],
 'CAKEBRL': [69.89735695167825, 1.0025438461227516],
 'GMTBRL': [1.3420688466475563, 1.0128821484132502],
 'HOTBRL': [0.0384562397491315, 1.0254997266435069],
 'APEBRL': [22.32100992764064, 1.0072657909585128],
 'APTBRL': [39.9724368140811, 1.0053429782213557]}

In [78]:
verify_list = []
for i in range(10):
    d = dt.datetime.now()
    verify_buy = pred_eval(model, epochs = 500, ticker_list = symbols, print_info = True)
    verify_list.append(verify_buy)
    d2 = dt.datetime.now()
    print(d2-d, '\n\n')
    
gambles = refine_gambles(verify_list)
gambles

1   BTCBRL     - LastPrice: 359839.0   / Pred1: 354704.188 / Pred2: 349642.844 || ActStatus: 0 / MyActStatus: 0 / MyGamble: 359839.0   / GambleProft:      
2   USDTBRL    - LastPrice: 5.021      / Pred1: 5.016      / Pred2: 5.011      || ActStatus: 0 / MyActStatus: 0 / MyGamble: 5.021      / GambleProft:      
3   ETHBRL     - LastPrice: 20317.9    / Pred1: 20496.105  / Pred2: 20675.777  || ActStatus: 1 / MyActStatus: 1 / MyGamble: 20475.609  / GambleProft: 1.008
4   BNBBRL     - LastPrice: 2672.0     / Pred1: 2669.465   / Pred2: 2666.938   || ActStatus: 0 / MyActStatus: 0 / MyGamble: 2672.0     / GambleProft:      
5   BUSDBRL    - LastPrice: 4.991      / Pred1: 5.004      / Pred2: 5.016      || ActStatus: 1 / MyActStatus: 1 / MyGamble: 4.999      / GambleProft: 1.002
6   XRPBRL     - LastPrice: 3.48       / Pred1: 3.459      / Pred2: 3.438      || ActStatus: 0 / MyActStatus: 0 / MyGamble: 3.48       / GambleProft:      
7   LINKBRL    - LastPrice: 105.85     / Pred1: 103.657    / Pre

{'HOTBRL': [0.04108096819122633, 1.0954924850993688],
 'ETCBRL': [203.7709503173828, 1.0067734699475435],
 'SOLBRL': [773.7625122070312, 1.0046254378174906],
 'AXSBRL': [38.70345115661621, 1.008953366960798],
 'C98BRL': [1.3190857805204392, 1.0084753673703664],
 'SANTOSBRL': [20.242926461356028, 1.0121463230678014],
 'SANDBRL': [1.8270456592241924, 1.011093336593355],
 'APEBRL': [22.351516723632812, 1.0086424514274734],
 'GALBRL': [8.726127624511719, 1.0053142424552672],
 'DOGEBRL': [0.8922253423080444, 1.0113640243800095],
 'ADABRL': [3.86464516544342, 1.0103647491355345],
 'CHZBRL': [0.8265880496580941, 1.0137209340913589],
 'BTTBRL': [0.015769243597754407, 1.0154052542018293],
 'ENJBRL': [2.0188918524115427, 1.5202498888641134]}

# Make a dataframe and save results

In [70]:
import threading
import multiprocessing
import os

# Function to execute in threads or processes
def your_function():
    pred_eval(model_1, epochs = 5000, lb_list = [11], ticker_list = symbols[:3])

def run_multithreading():
    num_threads = os.cpu_count()  # Use all available CPU cores
    threads = []
    for _ in range(num_threads):
        thread = threading.Thread(target=your_function)
        thread.start()
        threads.append(thread)
    for thread in threads:
        thread.join()

def run_multiprocessing():
    num_processes = os.cpu_count()  # Use all available CPU cores
    processes = []
    for _ in range(num_processes):
        process = multiprocessing.Process(target=your_function)
        process.start()
        processes.append(process)
    for process in processes:
        process.join()
        
your_function()

KeyboardInterrupt: 

In [ ]:
#csv_file = df.to_csv(f"Prediction_{present_day}.csv",index=False)

In [ ]:
dic = {"Ticker":[],"Act":[],"LastPrice":[],"Pred1":[],"Pred2":[]}
for ticker, status, last_price, pred_1, pred_2 in zip(symbols, pred_status_list, last_price_list, pred_1_list, pred_2_list):
    dic["Ticker"].append(ticker)
    dic["Act"].append(status)
    dic["LastPrice"].append(last_price)
    dic["Pred1"].append(pred_1)
    dic["Pred2"].append(pred_2)
df = pd.DataFrame(dic)
display(df)
display(df.loc[df["Act"]==True])

In [ ]:
pred_status_list, true_status_list = [], []
last_price_list = []
pred_1_list, pred_2_list = [], []
total_last_stock_price, total_pred_stock_price, total_true_stock_price = 0, 0, 0

for ticker in symbols:
    price = database[ticker]
    status, last_price, pred1, pred2, true_price1, true_price2 = predict(price, ticker=ticker)
    pred_status_list.append(status)
    last_price_list.append(last_price)
    pred_1_list.append(pred1)
    pred_2_list.append(pred2)
    pred_proft, true_proft = pred1 - last_price, true_price1 - last_price
    if status:
        total_last_stock_price += last_price
        total_pred_stock_price += pred1
        total_true_stock_price += true_price1
    if true_proft > 0:
        true_status_list.append(True)
    if true_proft <= 0:
        true_status_list.append(False)

print('\nPred proft %:', total_pred_stock_price / total_last_stock_price)
print('\nTrue proft %:', total_true_stock_price / total_last_stock_price)
my_confusion_matrix(true_status_list, pred_status_list)

In [50]:
max_crypto = ''
min_crypto = ''
max_max_value = 0
min_max_value = 1e3
for ticker in symbols:
    price = database[ticker]
    max_value = max(price)
    if max_value > max_max_value:
        max_max_value = max_value
        max_crypto = ticker
    if max_value < min_max_value:
        min_max_value = max_value
        min_crypto = ticker
        
print('MaxCrypto:', max_crypto, max_max_value)
print('MinCrypto:', min_crypto, min_max_value)

MaxCrypto: BTCBRL 376520.0
MinCrypto: SHIBBRL 0.00044702
